In [1]:
# apple_samsung_docket_2012_630.py
import os, time, requests
from typing import Optional, Dict, Any, List


In [2]:

BASE = "https://www.courtlistener.com/api/rest/v4"
TOKEN = "4cd64f7c2c009657cfa3133893d1cd846cfa7050"
if not TOKEN:
    raise SystemExit("Set COURTLISTENER_TOKEN in your environment.")
H = {"Authorization": f"Token {TOKEN}"}

def get_json(url: str, retries: int = 3, backoff: float = 1.0) -> Dict[str, Any]:
    for attempt in range(retries):
        try:
            r = requests.get(url, headers=H, timeout=60)
            if r.status_code in (502, 503, 504):  # transient upstream
                raise requests.HTTPError(f"{r.status_code} upstream")
            r.raise_for_status()
            return r.json()
        except requests.RequestException as e:
            if attempt == retries - 1:
                raise
            time.sleep(backoff * (2 ** attempt))
    raise RuntimeError("unreachable")

def find_docket_by_number(court: str, docket_number: str) -> Optional[Dict[str, Any]]:
    # Filter by court + exact docket number
    # Tip: docket_number is stored normalized by CourtListener; this exact string works for this case.
    url = f"{BASE}/dockets/?court={court}&docket_number={requests.utils.quote(docket_number)}"
    j = get_json(url)
    results = j.get("results", [])
    return results[0] if results else None

def fetch_docket_entries(docket_id: int, limit_pages: int = 200) -> List[Dict[str, Any]]:
    url = f"{BASE}/docket-entries/?docket={docket_id}&order_by=date_filed"
    out, pages = [], 0
    while url and pages < limit_pages:
        j = get_json(url)
        out.extend(j.get("results", []))
        url = j.get("next")
        pages += 1
        time.sleep(0.2)
    return out




In [ ]:
if __name__ == "__main__":
    COURT = "cand"                          # N.D. Cal.
    NUMBER = "5:12-cv-00630"                # Apple v. Samsung (2012 case)

    print(f"Looking up docket {NUMBER} in court {COURT} …")
    docket = find_docket_by_number(COURT, NUMBER)
    if not docket:
        raise SystemExit("❌ Docket not found. Double-check court code and docket number.")

    print("✅ Found docket:")
    print(f"  id:           {docket['id']}")
    print(f"  case_name:    {docket.get('case_name')}")
    print(f"  docket_num:   {docket.get('docket_number')}")
    print(f"  court:        {docket.get('court')}")
    print(f"  url:          https://www.courtlistener.com{docket.get('absolute_url','')}")

    print("\nFetching docket entries …")
    entries = fetch_docket_entries(docket["id"])
    print(f"✅ Retrieved {len(entries)} entries")

    # Show a few sample rows
    for i, de in enumerate(entries[:5], 1):
        desc = de.get("description") or de.get("entry_text") or "(no description)"
        date = de.get("date_filed") or "unknown"
        print(f"\nEntry {i}:")
        print(f"  date_filed:  {date}")
        description = desc[:200].replace('\\n',' ')
        print(f"""  description: {description}""")

    # If you want nested RECAP documents (often includes plain text):
    def extract_text_snippets(entry: Dict[str, Any], max_docs: int = 2) -> List[str]:
        recaps = entry.get("recap_documents") or entry.get("recap_document") or []
        if isinstance(recaps, dict):
            recaps = [recaps]
        out = []
        for rd in recaps[:max_docs]:
            txt = rd.get("plain_text") or ""
            if txt:
                out.append(txt[:400])
        return out

    # Example: show text snippets (if available) from the first entry with RECAP docs
    for de in entries:
        snippets = extract_text_snippets(de)
        if snippets:
            print("\n🔎 Sample RECAP text snippet:")
            print(snippets[0])
            break

SyntaxError: f-string expression part cannot include a backslash (2803797826.py, line 27)